<a href="https://colab.research.google.com/github/sarikasea/Streamlit-Interactive-Dashboards/blob/main/Streamlit_Upload_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit ngrok plotly pandas pyngrok


In [2]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np
import plotly.express as px

st.set_page_config(layout="wide")
st.title("Interactive Pandas + Plotly Streamlit demo (Colab)")

add_option = st.radio("How do you want to load data?", ("Random", "Upload"))
if add_option == "Random":
    df = pd.DataFrame(np.random.randn(100, 3), columns=["A", "B", "C"])
else:
    uploaded = st.file_uploader("Choose a CSV", type="csv")
    if uploaded:
        df = pd.read_csv(uploaded)
    else:
        st.stop()
st.dataframe(df)

st.header("Plotly: Scatter Plot")
x_axis = st.selectbox("X Axis", df.columns)
y_axis = st.selectbox("Y Axis", df.columns)
fig = px.scatter(df, x=x_axis, y=y_axis)
st.plotly_chart(fig)


Overwriting streamlit_app.py


In [3]:
import os
from pyngrok import ngrok
import threading
import time

from google.colab import userdata

# Set ngrok authtoken (must be in Colab Secrets)
ngrok_authtoken = userdata.get('NGROK_AUTHTOKEN')
if ngrok_authtoken is None:
    raise ValueError("NGROK_AUTHTOKEN not found in Colab Secrets. Please add it.")
os.environ["NGROK_AUTHTOKEN"] = ngrok_authtoken

# Threaded Streamlit runner
def run():
    os.system("streamlit run streamlit_app.py --server.port 8501 --server.enableCORS false --server.enableXsrfProtection false --browser.gatherUsageStats false")

thread = threading.Thread(target=run)
thread.start()

time.sleep(5)  # Wait for Streamlit to launch

# Start ngrok tunnel and print URL
public_url = ngrok.connect(8501).public_url
print(f"Streamlit app public URL: {public_url}")


Streamlit app public URL: https://9b6138f6086f.ngrok-free.app
